In [17]:
import tensorflow as tf
import os

base_dir = './train'

In [18]:

IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator( 
    rescale = 1./255, 
    validation_split=0.2    
)
train_generator = datagen.flow_from_directory( 
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),  
    batch_size = BATCH_SIZE, 
    subset='training'
)
val_generator = datagen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 289 images belonging to 36 classes.
Found 71 images belonging to 36 classes.


In [19]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'apple': 0, 'banana': 1, 'beetroot': 2, 'bell pepper': 3, 'cabbage': 4, 'capsicum': 5, 'carrot': 6, 'cauliflower': 7, 'chilli pepper': 8, 'corn': 9, 'cucumber': 10, 'eggplant': 11, 'garlic': 12, 'ginger': 13, 'grapes': 14, 'jalepeno': 15, 'kiwi': 16, 'lemon': 17, 'lettuce': 18, 'mango': 19, 'onion': 20, 'orange': 21, 'paprika': 22, 'pear': 23, 'peas': 24, 'pineapple': 25, 'pomegranate': 26, 'potato': 27, 'raddish': 28, 'soy beans': 29, 'spinach': 30, 'sweetcorn': 31, 'sweetpotato': 32, 'tomato': 33, 'turnip': 34, 'watermelon': 35}


In [20]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [21]:
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(36, activation='softmax')
])

In [22]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

Epoch 1/10
5/5 [==============================] - 27s 4s/step - loss: 3.7687 - accuracy: 0.0864 - val_loss: 2.7467 - val_accuracy: 0.1690
Epoch 2/10
5/5 [==============================] - 18s 3s/step - loss: 2.2189 - accuracy: 0.3796 - val_loss: 2.2006 - val_accuracy: 0.3803
Epoch 3/10
5/5 [==============================] - 19s 4s/step - loss: 1.4261 - accuracy: 0.5529 - val_loss: 1.6947 - val_accuracy: 0.5211
Epoch 4/10
5/5 [==============================] - 21s 4s/step - loss: 0.9445 - accuracy: 0.7393 - val_loss: 1.4476 - val_accuracy: 0.6056
Epoch 5/10
5/5 [==============================] - 19s 4s/step - loss: 0.6017 - accuracy: 0.8558 - val_loss: 1.0934 - val_accuracy: 0.7465
Epoch 6/10
5/5 [==============================] - 17s 3s/step - loss: 0.3650 - accuracy: 0.9475 - val_loss: 1.0125 - val_accuracy: 0.7324
Epoch 7/10
5/5 [==============================] - 16s 3s/step - loss: 0.2228 - accuracy: 0.9826 - val_loss: 0.9732 - val_accuracy: 0.7324
Epoch 8/10
5/5 [==================

In [24]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('veg_ml/assets/model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: assets
INFO:tensorflow:Assets written to: assets
